In [8]:
import os
import numpy as np

In [9]:
import mir_eval

def compute_scores(magenta_transcription, out_8v_thres,instrument_index):

    onsets_eval_magenta = np.arange(magenta_transcription.shape[0])*magenta_transcription[:,instrument_index]*0.01 
    onsets_eval_magenta = np.sort(onsets_eval_magenta[onsets_eval_magenta!=0])

    onsets_eval_model = np.arange(out_8v_thres.shape[0])*out_8v_thres[:,instrument_index]*0.01
    onsets_eval_model = np.sort(onsets_eval_model[onsets_eval_model!=0])


    #print(onsets_eval_magenta[0:5])
    #print(np.round(onsets_eval_model[0:5],2))
    #print(len(onsets_eval_model),len(onsets_eval_magenta))

    # mir_eval: 
    # Onsets should be provided in the form of a 1-dimensional array of onset 
    # times in seconds in increasing order.
    # default window is 0.05 seconds !! CHOICE OF WINDOW IS CRUCIAL
    # same value as in paper
    mir_eval.onset.validate(onsets_eval_magenta,onsets_eval_model)
    scores = mir_eval.onset.f_measure(onsets_eval_magenta,onsets_eval_model, window=0.5)
    
    return scores

In [18]:
eval_model_dir = 'eval_model'
if not os.path.exists(eval_model_dir):
    os.mkdir(eval_model_dir)

selected_models = [291, 295]
results_dir = 'results'

# list files in results folders
computed_files = os.listdir(os.path.join(results_dir, str(selected_models[0]) ))
print(computed_files)

num_instruments = 8
print(num_performances)

results/291
['291_drummer5-eval_session-5_funk-groove5_84_beat_4-4.json', '291_drummer7-session1-18_hiphop_100_beat_4-4.json', '291_drummer7-session1-14_jazz_100_beat_4-4.json', '291_drummer3-session2-12_rock_100_beat_4-4.json', '291_drummer8-session1-13_latin_118_beat_4-4.json', '291_drummer7-session3-22_pop-soft_83_beat_4-4.json', '291_drummer8-session1-14_hiphop_94_beat_4-4.json', '291_drummer8-session1-6_funk_80_beat_4-4.json', '291_drummer5-session1-10_latin-brazilian-sambareggae_96_beat_4-4.json', '291_drummer7-session2-97_pop_142_beat_4-4.json', '291_drummer5-session1-8_latin-venezuelan-merengue_162_beat_5-8.json', '291_drummer7-session3-67_neworleans-funk_93_beat_4-4.json', '291_drummer1-session3-6_dance-disco_120_beat_4-4.json']
444


In [26]:
f = os.path.join(os.path.join(results_dir, str(selected_models[0]) ),
                 '291_drummer5-eval_session-5_funk-groove5_84_beat_4-4.json')

performance_name = '_'.join(f.split('_')[1:])
print(performance_name)


for model in selected_models:
    eval_model_dir = os.path.join(eval_model_dir, str(model))




drummer5-eval_session-5_funk-groove5_84_beat_4-4.json


In [12]:
selected_models = [291, 295]
    
for model in selected_models:
    
    # path for storing model results
    model_dir = os.path.join(eval_model_dir, str(model))
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
        
        
    
    
    for performance_idx in range(num_performances):
    
        """
        # dev
        #if performance_idx>0:
        #    break
        
        # print percentage done
        percentage = (performance_idx+1) * 100  / num_performances
        if percentage % 10 == 0:
            print(str(percentage) + '%')
    
        # get audio and midi file path
        audio_file = os.path.join(groove_path, audio_files_list[performance_idx])
        midi_file = os.path.join(groove_path, audio_files_list[performance_idx].split('.')[-2] + '.mid')

    
        # model_results_json_path
        results_json_path = os.path.join(model_dir,'-'.join(audio_file.split('.')[-2].split('/')[1:])+'.json')
    
        # if csv file already exists do not compute again
        if not os.path.isfile(results_json_path):
            # get magenta onsets 
            magenta_onsets = get_magenta_onsets(midi_file)
            # get model onsets
            model_onsets = get_model_onsets(audio_file,model=model)
            results = {'magenta_onsets': magenta_onsets,
                       'model_onsets': model_onsets}
        
            with open(results_json_path, "w") as outfile:  
                json.dump(results, outfile,default=default) 
        """
        
        csv_eval_path =  os.path.join(eval_model_dir,
                                      'eval-'+'-'.join(audio_file.split('.')[-2].split('/')[1:])+'.json')
            
        # compute scores for each threshold_candidate and instrument and store them in dataframe
        df = pd.DataFrame(columns=['threshold','f_measure','precision','recall','instrument_idx'])
        
        for instrument_idx in range(num_instruments):       
            out_8v_thres = np.array(model_onsets)
            out_8v_thres[out_8v_thres < thres[instrument_idx]] = 0
            out_8v_thres[out_8v_thres > 0] = 1
            f_measure,precision,recall = compute_scores(magenta_transcription, out_8v_thres,instrument_idx)
            
            df = df.append({'threshold':thres,
                            'f_measure':f_measure,
                            'precision':precision,
                            'recall':recall,
                            'instrument_idx':instrument_idx
                           }, ignore_index=True)   

        # remove rows with instruments that do not appear
        df = df.replace(0, np.nan)
        df = df.dropna()
        idx = df.groupby(['instrument_idx'])['f_measure'].transform(max) == df['f_measure']
        df = df[idx].set_index('instrument_idx')
    
        # store data to csv
        df.to_csv(csv_path, index = True)

NameError: name 'audio_file' is not defined